# TP 1 PHY3051 - Ajustement d'une droite à des données
Ajuster une droite à des données est un peu le "hello world" de l'analyse de données.
On peut donc faire un exemple simple aujourd'hui.

## Simulation d'un jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

In [ ]:
# "Nouvelle" interface pour les nombres aléatoires avec Numpy
rng = np.random.default_rng(seed=3051)

In [ ]:
def linear_model(p: np.ndarray[float], x: np.ndarray[float]) -> np.ndarray[float]:
    m, b = p
    return m * x + b

In [ ]:
N = 100
m_true, b_true = 6, 2
p_true = np.array([m_true, b_true])
noise_level = 2

x = np.sort(rng.uniform(0, 10, size=N))
y_true = linear_model(p_true, x)

# Bruit gaussien indépendant avec barres d'erreur uniformes
yerr = noise_level * np.ones_like(x)
y = y_true + yerr * rng.standard_normal(N)

plt.plot(x, y_true, label="True signal")
plt.errorbar(x, y, yerr=yerr, fmt="k.", label="Data")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()

In [ ]:
def chi2_fun(
    p: np.ndarray[float],
    x: np.ndarray[float],
    y: np.ndarray[float],
    yerr: np.ndarray[float],
) -> float:
    y_mod = linear_model(p, x)
    return np.sum((y - y_mod)**2 / yerr**2)

In [ ]:
p_guess = np.array([5, 6])

plt.plot(x, linear_model(p_guess, x), label="Initial guess")
plt.errorbar(x, y, yerr=yerr, fmt="k.", label="Data")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()

In [ ]:
opt_res = minimize(chi2_fun, p_guess, args=(x, y, yerr))

In [ ]:
p_fit = opt_res.x
best_mod = linear_model(p_fit, x)
res = y - best_mod

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 8), sharex=True, gridspec_kw={"height_ratios": (2, 1)})
axes[0].plot(x, linear_model(p_guess, x), label="Initial guess")
axes[0].plot(x, best_mod, label="$\chi^2$ Best Fit")
axes[0].errorbar(x, y, yerr=yerr, fmt="k.", label="Data")
axes[0].set_ylabel("Y")
axes[0].legend()

axes[1].errorbar(x, res, yerr=yerr, fmt="k.")
axes[1].set_ylabel("Residuals")

axes[-1].set_xlabel("X")
plt.show()

In [ ]:
# Increase N pour voir mieux
plt.hist(res)
plt.show()

In [ ]:
# Save the output
np.savetxt("simulated_data.txt", np.vstack([x, y, yerr]).T)
np.savetxt("best_mod_y.txt", best_mod)